In [1]:
import requests
import time
from datetime import datetime
import pandas as pd
import os
import json
import uuid
import sqlite3
import sys
from dotenv import load_dotenv
# Load the .env file
load_dotenv()
# Access the environment variables
project_folder_path = os.getenv('PROJECT_FOLDER_PATH')
print(project_folder_path)
project_folder_path = '/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database'
sys.path.append(project_folder_path)
from utils.db_functs import *
from utils.requests_functs import *
from utils.html_functs import *
from utils.pdf_functs import *

/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roruizf/anaconda3/envs/data_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


/mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database


### Database and directories

In [2]:
db_file_name = 'cev-database-coordinates-v1.db'
db_file_path = os.path.join(project_folder_path, 'data', 'sqlite', db_file_name)

## 1) Define parameters to run a `search`
This can be a new search or retake a previous one

In [3]:
#search_id = '0f62f13c-2f06-43f9-b10c-323673190d29'
search_id = '010576e8-90f1-42c4-88bc-3965b5c4c238'
#search_date = '2024-05-28'
search_date = '2024-04-21'

## 2) Define a dataframe with download coordinates
The dataframe must contain 1 row per `comuna` meaning 348 rows in total.

In [4]:
query = f"""
        SELECT * FROM evals_pdf_reports_download_coordinates
        WHERE search_id = "{search_id}";
        """
print(query)


        SELECT * FROM evals_pdf_reports_download_coordinates
        WHERE search_id = "010576e8-90f1-42c4-88bc-3965b5c4c238";
        


In [5]:
df = create_dataframe_from_query(db_file_path, query)
# Drop specified columns
columns_to_drop = ['id']
df = df.drop(columns=columns_to_drop)
df.head()

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,fec88ef2-ceb6-5143-850c-473800d9bcfa,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,g1C7GRvPNukp6JH4Vpy/K/3+zUISmSs+Eg2LE8BMV74A8v...,Page$1
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,81f198e3-1fb7-504a-a0cd-8b8b21b74a7e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,gd/q5M/ujufbV9MSWV+MrW0VklfDBf+sczGGZOg9ihQJx+...,Page$2
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,41415be6-e4c5-565a-a85e-01d2fe8c1e6d,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,RNIiIK0dFhwfleaBCk5tcTuMHx9ElJwANpfgDvFZXv4Pbm...,Page$3
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,35096c96-d248-5926-9b44-3d3476978ec9,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,5y0b8PMBBAUUc5f2cj2GyFNCwrX3VMcmYldeGyE3SHXx6Q...,Page$4
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,04887b1c-b82b-5a63-9a37-34cc42cff85b,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,iFxRVXi6rMfK02bhUpzFnsybOS7OQHj7LCYTdrvpy1jMOR...,Page$5


Set additional features to track the status of downloads as well as times and file names

In [6]:
df['status'] = None
df['pdf_filename'] = None
df['downloaded_at'] = None
df.head()

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename,downloaded_at
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,fec88ef2-ceb6-5143-850c-473800d9bcfa,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,g1C7GRvPNukp6JH4Vpy/K/3+zUISmSs+Eg2LE8BMV74A8v...,Page$1,None,None,None
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,81f198e3-1fb7-504a-a0cd-8b8b21b74a7e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,gd/q5M/ujufbV9MSWV+MrW0VklfDBf+sczGGZOg9ihQJx+...,Page$2,None,None,None
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,41415be6-e4c5-565a-a85e-01d2fe8c1e6d,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,RNIiIK0dFhwfleaBCk5tcTuMHx9ElJwANpfgDvFZXv4Pbm...,Page$3,None,None,None
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,35096c96-d248-5926-9b44-3d3476978ec9,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,5y0b8PMBBAUUc5f2cj2GyFNCwrX3VMcmYldeGyE3SHXx6Q...,Page$4,None,None,None
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,04887b1c-b82b-5a63-9a37-34cc42cff85b,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,iFxRVXi6rMfK02bhUpzFnsybOS7OQHj7LCYTdrvpy1jMOR...,Page$5,None,None,None


## 3) Check first if any pdf files has been downloaded
This is useful in case a seach has been run and has not completed

### 3.1) Read all html files for the corresponding `search_date` and `search_id`

In [7]:
# Define the directory path
pdf_files_dir = os.path.join(project_folder_path, 'data', 'raw', '3_evals_reports', 'pdf_files')

# Check if the directory exists
if not os.path.exists(pdf_files_dir):
    # If it doesn't exist, create the directory
    os.makedirs(pdf_files_dir)
    print(f"Directory '{pdf_files_dir}' created.")

# List all files in the directory
pdf_file_paths = find_pdf_files(pdf_files_dir)
print(f'The directory: {pdf_files_dir} contains {len(pdf_file_paths)} out of {df.shape[0]}')

The directory: /mnt/c/Users/rober/OneDrive/8_DEVELOPMENT/cev-database/data/raw/3_evals_reports/pdf_files contains 12363 out of 7300


### 3.2) Fill on dataframe all those rows corresponding to files already downloaded

In [8]:
for pdf_file_path in pdf_file_paths:
    pdf_file_name = os.path.split(pdf_file_path)[-1]
    pdf_file_list = pdf_file_name.split('_')
    region_id = int(pdf_file_list[0])
    comuna_id = int(pdf_file_list[1])
    tipo_evaluacion = int(pdf_file_list[2])
    eval_id = pdf_file_list[-1].split('.')[0]
    row = df[df['region_id'].astype(str) + '_' + df['comuna_id'].astype(str) + '_' + df['tipo_evaluacion'].astype(str)+ '_' + df['eval_id'].astype(str) == str(region_id) + '_' + str(comuna_id) + '_' + str(tipo_evaluacion) + '_' + str(eval_id)]
    if not row.empty:
        df.loc[row.index, 'status'] = 'Successful'
        df.loc[row.index, 'pdf_filename'] = pdf_file_name
        #df.loc[row.index, 'downloaded_at'] = None    
df.dropna()

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename,downloaded_at


In [9]:
df[~df['pdf_filename'].isna()]

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename,downloaded_at
0,5,1,1,1,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,fec88ef2-ceb6-5143-850c-473800d9bcfa,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,g1C7GRvPNukp6JH4Vpy/K/3+zUISmSs+Eg2LE8BMV74A8v...,Page$1,Successful,1_5_1_fec88ef2-ceb6-5143-850c-473800d9bcfa.pdf,None
1,5,1,1,2,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,81f198e3-1fb7-504a-a0cd-8b8b21b74a7e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,gd/q5M/ujufbV9MSWV+MrW0VklfDBf+sczGGZOg9ihQJx+...,Page$2,Successful,1_5_1_81f198e3-1fb7-504a-a0cd-8b8b21b74a7e.pdf,None
2,5,1,1,3,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,41415be6-e4c5-565a-a85e-01d2fe8c1e6d,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,RNIiIK0dFhwfleaBCk5tcTuMHx9ElJwANpfgDvFZXv4Pbm...,Page$3,Successful,1_5_1_41415be6-e4c5-565a-a85e-01d2fe8c1e6d.pdf,None
3,5,1,1,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,35096c96-d248-5926-9b44-3d3476978ec9,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,5y0b8PMBBAUUc5f2cj2GyFNCwrX3VMcmYldeGyE3SHXx6Q...,Page$4,Successful,1_5_1_35096c96-d248-5926-9b44-3d3476978ec9.pdf,None
4,5,1,1,5,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,04887b1c-b82b-5a63-9a37-34cc42cff85b,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,iFxRVXi6rMfK02bhUpzFnsybOS7OQHj7LCYTdrvpy1jMOR...,Page$5,Successful,1_5_1_04887b1c-b82b-5a63-9a37-34cc42cff85b.pdf,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7270,294,13,1,48,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,1ddbf4f2-097b-5e77-8c41-fd3071486a7e,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,5YsLH0+DYzSmpWB87+L+Ivd1bvmUDouPaZrXCf9l+OfYk7...,Page$48,Successful,13_294_1_1ddbf4f2-097b-5e77-8c41-fd3071486a7e.pdf,None
7276,294,13,1,54,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,9de06c28-5668-5f72-8886-f4e1a4a9c91a,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,GUD+23eFIy7LETf8q8dDTSPDzBmyOsAEgts+iGfib/lI5a...,Page$54,Successful,13_294_1_9de06c28-5668-5f72-8886-f4e1a4a9c91a.pdf,None
7287,294,13,1,65,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ccc9a25a-9b8b-566d-86b8-2e1c82f4d230,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,tEU1xNce8wxoW1fbGHul4eJUcUGNBjxLwnXTDArgWyrs5w...,Page$65,Successful,13_294_1_ccc9a25a-9b8b-566d-86b8-2e1c82f4d230.pdf,None
7293,5,1,2,4,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,fe53c8ac-3fcf-5168-9948-9b93e8cffe5b,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,ctl00$ContentPlaceHolder1$grdViviendasCal$ctl0...,Wd3shf11mlyq++gTRf64C688TBNW3hfwVDPzsrHJpdhneP...,Page$4,Successful,1_5_2_fe53c8ac-3fcf-5168-9948-9b93e8cffe5b.pdf,None


In [10]:
df['pdf_filename'].isna().value_counts(dropna=False)

pdf_filename
True     6652
False     648
Name: count, dtype: int64

### Download pdf files: 1 file per evaluacion

In [11]:
HOME_URL = 'http://calificacionenergeticaweb.minvu.cl/Publico/BusquedaVivienda.aspx'

In [12]:
# Shuffle the DataFrame
df = df[df['status'] != 'Successful']
df = df.sample(frac=1, random_state=42)  # random_state for reproducibility
df = df.sort_values(by='pagina', ascending=False).reset_index(drop=True)
df = df.head(20)
df

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename,downloaded_at
6366,187,13,1,378,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,aacf41db-6b4e-5fb3-8710-dba1b88c9f7f,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,Pqm0T7iF8k8uBG9J0APBk9mOP3DIY/IYHGlVbZslJ1v79v...,Page$378,None,None,None
6365,187,13,1,377,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,b0c6e0ec-af60-5405-bfb1-4ec55689b1b2,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,izAl31SqTvgvE43qY12gQQ6SxOJZ8nZ+w3Pzs9kIPBfOQ0...,Page$377,None,None,None
6364,187,13,1,376,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,1edbd17a-3c12-53d8-84fc-992de87fb66f,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,eqzrx/gDPy6bHRZeG9QnR65GlyL31lJYq+HG00pP7V6Rmd...,Page$376,None,None,None
6363,187,13,1,375,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,a51f5349-4022-55be-9980-35fe6706161c,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,8B35iEE9JyQGDRiETBsDpgJJEpJX0JOBXPfNOclYP4fa13...,Page$375,None,None,None
6362,187,13,1,374,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,bb263304-45b6-56fd-9302-538b98756f03,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,PFKvBizKrVj6OdqROKyHGp6ZmJWqe1N2XT8dHIckwHcpvC...,Page$374,None,None,None
6361,187,13,1,373,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,f29acf8a-72b0-5eca-81a7-1dd885818114,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,znBf9uvry1LF3J22ueEltTMSUeykxARv4WKr58V1zuWghS...,Page$373,None,None,None
6360,187,13,1,372,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,8d62778e-f87c-5fe9-b7c2-8506004fe7d3,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,IBEMmrcheTgZbf+HPxOiqNMj3hT5MTDvUhXbS9zJ1CHpoM...,Page$372,None,None,None
6359,187,13,1,371,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,6aa04614-5a07-51d9-a406-271741da77aa,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,OEDJl6UTh+MuQYTN12UQ/9Y5x2qBO7AHTB2E0+intscca0...,Page$371,None,None,None
6358,187,13,1,370,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ccc2d2e9-af42-58fa-86b9-af4be4247071,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,OZdlqlcqgYPtVA5T9eYeM0KGpLQ7Yb5P9EB/Gb35xlyRJ7...,Page$370,None,None,None
6357,187,13,1,369,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,dc9e08bd-47c7-504b-8721-949b2c10d516,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,sc3fHoSo6j6UtuSnoa/Jm2fcf2jvLfu68mmw8C4/EHoJTT...,Page$369,None,None,None


In [13]:
MAX_ATTEMPTS = 5
DELAY_BETWEEN_ATTEMPTS = 15

attempts = 0
while attempts < MAX_ATTEMPTS:
    all_successful = True  # Flag to track if all rows are successful in this attempt
    communes_to_process = df[df['status'] != 'Successful'].shape[0]
    print(f"Attempt {attempts + 1} of {MAX_ATTEMPTS}: {communes_to_process} communes to process")

    for index, row in df.iterrows():
        if df.loc[index, 'status'] != 'Successful':
            region_id = str(row['region_id'])
            comuna_id = str(row['comuna_id'])
            tipo_evaluacion = str(row['tipo_evaluacion'])
            #eventtarget = row['eventtarget']
            eventargument = 'Page$' + str(row['pagina'])
            viewstate = row['viewstate']
            target_report_label_code = row['codigo_informe']
            # eventtarget
            if str(tipo_evaluacion) == '1':
                eventtarget = 'ctl00$ContentPlaceHolder1$grdViviendasPre'
            else:
                eventtarget = 'ctl00$ContentPlaceHolder1$grdViviendasCal'
            
            form_data_pdf = form_data_pdf_report(eventtarget, eventargument, viewstate, region_id, comuna_id, tipo_evaluacion, target_report_label_code)
            
            print(f" * Processing {region_id}, {comuna_id}: File {index+1} out of {df.shape[0]}")
           
            try:
                response = requests.post(HOME_URL, data=form_data_pdf)

                if response.status_code == 200:
                    # Record the current time after making the request
                    request_time = datetime.now()
                    downloaded_on = request_time.strftime("%Y-%m-%d")
                    downloaded_at = request_time.strftime("%Y-%m-%d %H:%M:%S")
                    
                    df.loc[index, 'status'] = 'Successful'
                    
                    # Saving pdf data into a local folder
                    pdf_path = os.path.join(pdf_files_dir, region_id)
                   
                    if not os.path.exists(pdf_path):
                        os.makedirs(pdf_path)
                        
                    pdf_filename = region_id + "_" + comuna_id + "_" + tipo_evaluacion + "_" + row['eval_id'] + '.pdf'                    

                    with open(os.path.join(pdf_path, pdf_filename), "wb") as f:
                        f.write(response.content)                    
                    
                    df.loc[index, 'pdf_filename'] = downloaded_at
                    df.loc[index, 'downloaded_at'] = downloaded_at
                else:
                    df.loc[index, 'status'] = 'Failed'
                    print(f"Request failed for region {region_id}, commune {comuna_id}. Skipping to the next row.")
                    continue  # Skip to the next iteration of the loop
                #df.loc[index, 'status_code'] = response.status_code
            
            except Exception as e:
                print(f"An error occurred for region {region_id}, commune {comuna_id}: {str(e)}")
                df.loc[index, 'status'] = 'Failed'
                continue  # Skip to the next iteration of the loop
            
    
    all_successful = all_successful and (df['status'] == 'Successful').all()

    if all_successful:
        break  # Exit the loop if all rows are successful
    else:
        attempts += 1
        time.sleep(DELAY_BETWEEN_ATTEMPTS)  # Delay between attempts

Attempt 1 of 5: 20 communes to process
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187
 * Processing 13, 187


In [14]:
# Drop specified columns
columns_to_drop = ['codigo_etiqueta', 'eventargument', 'viewstate']
out_df = df#.drop(columns=columns_to_drop).dropna().reset_index(drop=True)
out_df

,comuna_id,region_id,tipo_evaluacion,pagina,search_id,search_date,eval_id,codigo_informe,codigo_etiqueta,viewstate,eventargument,status,pdf_filename,downloaded_at
6366,187,13,1,378,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,aacf41db-6b4e-5fb3-8710-dba1b88c9f7f,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,Pqm0T7iF8k8uBG9J0APBk9mOP3DIY/IYHGlVbZslJ1v79v...,Page$378,Successful,2024-06-02 10:00:14,2024-06-02 10:00:14
6365,187,13,1,377,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,b0c6e0ec-af60-5405-bfb1-4ec55689b1b2,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,izAl31SqTvgvE43qY12gQQ6SxOJZ8nZ+w3Pzs9kIPBfOQ0...,Page$377,Successful,2024-06-02 10:00:21,2024-06-02 10:00:21
6364,187,13,1,376,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,1edbd17a-3c12-53d8-84fc-992de87fb66f,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,eqzrx/gDPy6bHRZeG9QnR65GlyL31lJYq+HG00pP7V6Rmd...,Page$376,Successful,2024-06-02 10:00:27,2024-06-02 10:00:27
6363,187,13,1,375,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,a51f5349-4022-55be-9980-35fe6706161c,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,8B35iEE9JyQGDRiETBsDpgJJEpJX0JOBXPfNOclYP4fa13...,Page$375,Successful,2024-06-02 10:00:34,2024-06-02 10:00:34
6362,187,13,1,374,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,bb263304-45b6-56fd-9302-538b98756f03,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,PFKvBizKrVj6OdqROKyHGp6ZmJWqe1N2XT8dHIckwHcpvC...,Page$374,Successful,2024-06-02 10:00:41,2024-06-02 10:00:41
6361,187,13,1,373,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,f29acf8a-72b0-5eca-81a7-1dd885818114,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,znBf9uvry1LF3J22ueEltTMSUeykxARv4WKr58V1zuWghS...,Page$373,Successful,2024-06-02 10:00:47,2024-06-02 10:00:47
6360,187,13,1,372,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,8d62778e-f87c-5fe9-b7c2-8506004fe7d3,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,IBEMmrcheTgZbf+HPxOiqNMj3hT5MTDvUhXbS9zJ1CHpoM...,Page$372,Successful,2024-06-02 10:00:54,2024-06-02 10:00:54
6359,187,13,1,371,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,6aa04614-5a07-51d9-a406-271741da77aa,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,OEDJl6UTh+MuQYTN12UQ/9Y5x2qBO7AHTB2E0+intscca0...,Page$371,Successful,2024-06-02 10:01:01,2024-06-02 10:01:01
6358,187,13,1,370,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,ccc2d2e9-af42-58fa-86b9-af4be4247071,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,OZdlqlcqgYPtVA5T9eYeM0KGpLQ7Yb5P9EB/Gb35xlyRJ7...,Page$370,Successful,2024-06-02 10:01:08,2024-06-02 10:01:08
6357,187,13,1,369,010576e8-90f1-42c4-88bc-3965b5c4c238,2024-04-21,dc9e08bd-47c7-504b-8721-949b2c10d516,ctl00$ContentPlaceHolder1$grdViviendasPre$ctl0...,None,sc3fHoSo6j6UtuSnoa/Jm2fcf2jvLfu68mmw8C4/EHoJTT...,Page$369,Successful,2024-06-02 10:01:14,2024-06-02 10:01:14


In [15]:
#insert_unique_rows_from_dataframe(db_file_path, 'evals_html_downloaded_files', out_df, unique_columns=['comuna_id', 'region_id', 'tipo_evaluacion', 'pagina', 'search_id'])